# GD (Guess and Determine) optimization 2, it takes few hours to return the complete list of candidate states

In this second optimization, the task is to eliminate the recursion in the function by replacing it with 9 nested loops. The reasoning remains the same.

In [2]:
from GDop1 import *

def find_state2(keystream):
  """

    This is the initial optimization approach, and it approximately generates 2^16 candidate
    states. Each nested loop iterates through the possible values of the corresponding layer's
    state variable, following the steps outlined in the comments. If a guess is correct, it proceeds
    to the next layer with an updated state. Once all layers are iterated, the final state is passed
    to the test function for further verification against the remaining 23 observed keystream bits,
    which will narrow down the list of candidates to approximately 216 states that produced the observed 32-bit sample
    Args:
        keystream (list): List of observed keystream bits.

    Returns:
        list: List of approximately 2^16 candidate states.

    """
  # Iterate through possible values of the first layer's state variable
  for i0 in range(1<<bits[0]):
    state0 = expand(masks[0], i0)
    #Check if state0 produces the correct first keystream
    if f20(state0) != keystream[0]:
      continue

    # Iterate through possible values of the second layer's state variable
    for i1 in range(1<<(bits[1]+1)): # guess LFSR output bit 0
      state1 = (state0>>1) | expand(masks[1]|(1<<47), i1)
      #Check if state1 produces the correct second keystream
      if f20(state1) != keystream[1]:
        continue

      # Iterate through possible values of the third layer's state variable
      for i2 in range(1<<(bits[2]+1)): # guess LFSR output bit 1
        state2 = (state1>>1) | expand(masks[2]|(1<<47), i2)
        #Check if state2 produces the correct third keystream
        if f20(state2) != keystream[2]:
          continue

        # Iterate through possible values of the fourth layer's state variable
        for i3 in range(1<<bits[3]):
          state3 = lfsr(state2) | expand(masks[3], i3)
          #Check if state3 produces the correct fourth keystream
          if f20(state3) != keystream[3]:
            continue

          # Iterate through possible values of the fifth layer's state variable
          for i4 in range(1<<bits[4]):
            state4 = lfsr(state3) | expand(masks[4], i4)
            #Check if state4 produces the correct fifth keystream
            if f20(state4) != keystream[4]:
              continue

            # Iterate through possible values of the sixth layer's state variable
            for i5 in range(1<<bits[5]):
              state5 = lfsr(state4) | expand(masks[5], i5)
              #Check if state5 produces the correct sixth keystream
              if f20(state5) != keystream[5]:
                continue

              # Iterate through possible values of the seventh layer's state variable
              for i6 in range(1<<bits[6]):
                state6 = lfsr(state5) | expand(masks[6], i6)
                #Check if state6 produces the correct seventh keystream
                if f20(state6) != keystream[6]:
                  continue

                # Iterate through possible values of the eighth layer's state variable
                for i7 in range(1<<bits[7]):
                  state7 = lfsr(state6) | expand(masks[7], i7)
                  #Check if state7 produces the correct eighth keystream
                  if f20(state7) != keystream[7]:
                    continue

                  # Iterate through possible values of the ninth layer's state variable
                  for i8 in range(1<<bits[8]):
                    #Check if state8 produces the correct ninth keystream
                    state8 = lfsr(state7) | expand(masks[8], i8)
                    if f20(state8) != keystream[8]:
                      continue
                    #Test it against the remaining 23 observed keystream bits using test function
                    test(lfsr(state8),keystream)
    


To verify the validity of the function, we can proceed exactly as before.

We start by defining our key, uid, iv, keystream and the filt_mask=0x5806b4a2d16c

In [7]:
key, uid, iv = 0x414141414141, 0x42424242, 0x43434343
state = hitag2_init(0x414141414141, 0x42424242, 0x43434343)
keystream_int = hitag2(state,32)
keystream = list(map(int, "{0:032b}".format(keystream_int)))
mask_filt=0x5806b4a2d16c


Next, we calculate the first 9 evolutions of the register

In [10]:
test_states = []
for _ in range(len(masks)):
    test_states.append(state)
    state = lfsr(state)

And now we re-implement our function by adding the condition: if (new_state & filt_mask) != (test_states[layer] & filt_mask) to check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.

In [11]:
def find_state2_val(keystream):
  """

    This is the initial optimization approach, and it approximately generates 2^16 candidate
    states. Each nested loop iterates through the possible values of the corresponding layer's
    state variable, following the steps outlined in the comments. If a guess is correct, it proceeds
    to the next layer with an updated state. Once all layers are iterated, the final state is passed
    to the test function for further verification against the remaining 23 observed keystream bits,
    which will narrow down the list of candidates to approximately 216 states that produced the observed 32-bit sample
    Args:
        keystream (list): List of observed keystream bits.

    Returns:
        list: List of approximately 2^16 candidate states.

    """
  # Iterate through possible values of the first layer's state variable
  for i0 in range(1<<bits[0]):
    state0 = expand(masks[0], i0)
    #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
    if (state0 & mask_filt) != (test_states[0] & mask_filt):
            continue
    #Check if state0 produces the correct first keystream
    if f20(state0) != keystream[0]:
      continue

    # Iterate through possible values of the second layer's state variable
    for i1 in range(1<<(bits[1]+1)): # guess LFSR output bit 0
      state1 = (state0>>1) | expand(masks[1]|(1<<47), i1)
      #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
      if (state1 & mask_filt) != (test_states[1] & mask_filt):
            continue
      #Check if state1 produces the correct second keystream
      if f20(state1) != keystream[1]:
        continue

      # Iterate through possible values of the third layer's state variable
      for i2 in range(1<<(bits[2]+1)): # guess LFSR output bit 1
        state2 = (state1>>1) | expand(masks[2]|(1<<47), i2)
        #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
        if (state2 & mask_filt) != (test_states[2] & mask_filt):
            continue
        #Check if state2 produces the correct third keystream
        if f20(state2) != keystream[2]:
          continue

        # Iterate through possible values of the fourth layer's state variable
        for i3 in range(1<<bits[3]):
          state3 = lfsr(state2) | expand(masks[3], i3)
          #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
          if (state3 & mask_filt) != (test_states[3] & mask_filt):
              continue
          #Check if state3 produces the correct fourth keystream
          if f20(state3) != keystream[3]:
            continue

          # Iterate through possible values of the fifth layer's state variable
          for i4 in range(1<<bits[4]):
            state4 = lfsr(state3) | expand(masks[4], i4)
            #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
            if (state4 & mask_filt) != (test_states[4] & mask_filt):
                continue
            #Check if state4 produces the correct fifth keystream
            if f20(state4) != keystream[4]:
              continue

            # Iterate through possible values of the sixth layer's state variable
            for i5 in range(1<<bits[5]):
              state5 = lfsr(state4) | expand(masks[5], i5)
              #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
              if (state5 & mask_filt) != (test_states[5] & mask_filt):
                  continue
              #Check if state5 produces the correct sixth keystream
              if f20(state5) != keystream[5]:
                continue

              # Iterate through possible values of the seventh layer's state variable
              for i6 in range(1<<bits[6]):
                state6 = lfsr(state5) | expand(masks[6], i6)
                #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
                if (state6 & mask_filt) != (test_states[6] & mask_filt):
                  continue
                #Check if state6 produces the correct seventh keystream
                if f20(state6) != keystream[6]:
                  continue

                # Iterate through possible values of the eighth layer's state variable
                for i7 in range(1<<bits[7]):
                  state7 = lfsr(state6) | expand(masks[7], i7)
                  #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
                  if (state7 & mask_filt) != (test_states[7] & mask_filt):
                    continue
                  #Check if state7 produces the correct eighth keystream
                  if f20(state7) != keystream[7]:
                    continue

                  # Iterate through possible values of the ninth layer's state variable
                  for i8 in range(1<<bits[8]):
                    #Check if state8 produces the correct ninth keystream
                    state8 = lfsr(state7) | expand(masks[8], i8)
                    #check if the common bits (after applying a filter mask) between the guessed state and the real state are not different.
                    if (state8 & mask_filt) != (test_states[8] & mask_filt):
                      continue
                    if f20(state8) != keystream[8]:
                      continue
                    #Test it against the remaining 23 observed keystream bits using test function
                    print(test(lfsr(state8),keystream))

In [16]:
find_state2_val(keystream)

[True, 198129007952514]


We can see that the result returned by the function is correct.

In [14]:
198129007952514==hitag2_init(0x414141414141, 0x42424242, 0x43434343)

True